In [155]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from conct_db import enginex
from datetime import date

**LEER LOS DATOS DE ORIGIEN //CARTERA DE REGISTROS//**

In [160]:
data1 = pd.read_csv('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.csv', encoding='UTF-8')
df = pd.DataFrame(data1)
# df.loc[[2134]]
# df.sample(3)

,Código Distribuidor,Distribuidor,POC,Clientes,Registrado,Fecha registro,Adherible,Adherido,Fecha adhesion,CanjeL3M,Puntos Disponibles,COD_CLIENTE
3215,369117,1994 SRL,36911700010174.0,1.0,1.0,18/1/2022,1.0,1.0,18/1/2022,SI,1258.0,10174.0
2837,369117,1994 SRL,36911700009118.0,1.0,1.0,1/2/2022,1.0,1.0,1/2/2022,NO,614.0,09118.0
6583,369846,TOTAL CENTER S.R.L.,36984600006988.0,1.0,1.0,18/1/2022,1.0,1.0,28/1/2022,NO,853.0,06988.0


**TRANSFORMAR LOS DATOS PARA PODER CARGARLOS A LA BASE**

In [148]:
df['POC'] = df['POC'].astype(str)
# df['Fecha adhesion'] = df['Fecha adhesion'].convert_dtypes(int)
df['Fecha adhesion'] = pd.to_datetime(df['Fecha adhesion'],  format='%d/%m/%Y', errors='coerce')
df['Fecha registro'] = pd.to_datetime(df['Fecha registro'], format='%d/%m/%Y', errors='coerce', )
# df['Fecha adhesion'] = df['Fecha adhesion'].dt.strftime('%m/%d/%Y')

In [149]:
# df['COD_CLIENTE'] = df.POC.str[-5:]
df['COD_CLIENTE'] = df.POC.str.extract('(0{2,}\d+)')
df.rename({'Código Distribuidor':'COD_DISTRI'}, axis=1, inplace=True)
# df['COD_DISTRI'] = df.POC.str.extract('(\d{6,6}(?:\.\d+)?)')
df[['COD_CLIENTE','Registrado', 'Adherible', 'Adherido']] = df[['COD_CLIENTE','Registrado', 'Adherible', 'Adherido']].astype('Int64')
try:
    df.drop(columns=['POC', 'Distribuidor', 'Clientes'], inplace=True)
except:
    pass
df = df[['COD_DISTRI', 'COD_CLIENTE', 'Registrado', 'Fecha registro', 'Adherible', 'Adherido', 'Fecha adhesion', 'CanjeL3M', 'Puntos Disponibles']]
df.sample(3)

,COD_DISTRI,COD_CLIENTE,Registrado,Fecha registro,Adherible,Adherido,Fecha adhesion,CanjeL3M,Puntos Disponibles
2418,369117,7036,1,2022-02-02,1,1,2022-02-02,SI,801.0
3555,369117,11040,1,2021-12-22,1,1,2022-03-23,NO,1298.0
5378,369846,3191,1,2021-11-01,1,1,2021-11-01,SI,4698.0


In [153]:

try:
    df.to_excel('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.xlsx')
    df.to_sql('BEES_Cartera_Registros', con=enginex, if_exists='replace', index=False)
    print('Carga Exitosa')
except Exception as ex:
    print(ex)
    

No engine for filetype: 'xlsb'
